<a href="https://colab.research.google.com/github/FrancescoManca-FM/Project_MLDL/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision.datasets import CIFAR100
import numpy as np
import torch
from PIL import Image
import glob
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR


In [ ]:
total_classes = 100 # classes of CIFAR100
iterations = 10 
classes_per_iter = total_classes/iterations

# parameters 
batch_size = 128
lr = 0.01
decay = 0.0001
epochs = 70
momentum = 0.9
factor = 5
loss = nn.CrossEntropyLoss()

resNet18 = models.resnet18()

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [ ]:
#import train and test datasets
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train = CIFAR100("./data", train = True, download=True, transform=transform)
test = CIFAR100("./data", train= False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#counting rows of dataset
print(len(train), len(test))

50000 10000


In [ ]:
def incrementalFit(train_loader, valid_loader, network):
  optimizer = optim.SGD(network.parameters(), lr=lr, momentum=momentum, weight_decay=decay)
  scheduler = MultiStepLR(optimizer, [int(0.7*epochs), int(0.9*epochs)], factor)
  print("1")
  for epoch in range(epochs):
    network.train()
    scheduler.step()
    train_loss = 0
    train_correct = 0
    train_total = 0
    valid_loss = 0
    valid_correct = 0
    valid_total = 0
    print("1")
    # In each epoch, we do a full pass over the training data:
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        print("1")
        inputs = inputs.to(device)
        targets = targets.to(device)
        logits, feat = network.forward(inputs)  # feature vector only
        prediction = network.predict(logits)  # make the prediction
        loss_bx = loss(prediction, targets)  # CE loss
        loss_bx.backward()
        optimizer.step
        train_loss += loss_bx.item()
        _, predicted = prediction.max(1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets).sum().item
    train_acc = 100. * train_correct / train_tot
    print(train_acc)
    network.eval()
     # In each epoch, we do a full pass over the training data:
     #for inputs, targets in valid_loader:
#      #          inputs = inputs.to(device)
      #          targets = targets.to(device)
#
      #          logits, feat = network.forward(inputs)  # feature vector only
      #          prediction = network.predict(logits)  # make the prediction
#
      #          loss_bx = loss(prediction, targets)  # CE loss
#
      #          valid_loss += loss_bx.item()
      #          _, predicted = prediction.max(1)
      #          valid_total += targets.size(0)
      #          valid_correct += predicted.eq(targets).sum().item()
      #          
      #          valid_acc = 100*valid_correct/valid_total

# function to predict our train_loader and valid_loader
def test(loader, network):

  all_pred = []
  all_labels = []

  #compute prediction
  for inputs, targets in loader:
    outputs = network.forward(inputs)
    predictions = network.predict(outputs).cpu().detach() #return score classes as logits
    
    _, predicted = prediction.max(1)

    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()

    all_preds.extend(predicted.tolist())
    all_labels.extend(targets.data.tolist())
    return[100.*correct / total], all_preds, all_labels

#function to plot confusion matrix
def plotConfusionMatrix(confusionMatrixData, seed):
 fig,ax=plt.subplots(figsize=(10,10))
 sns.heatmap(confusionMatrixData,cmap='terrain',ax=ax)
 plt.ylabel('True label')
 plt.xlabel('Predicted label')
 plt.title("Confusion Matrix {} - seed: {}".format(method, seed))
 plt.savefig(filename, format='png', dpi=300)
 plt.show()




In [ ]:
#iterate over the dataset 10 times (dividing 100 classes in 10) 
test = [] #initialized here because we test over all the classes not only those one in which I train
for i in range(iterations):
  i = 0
  classes_current_iter = range(i*iterations, i*iterations+iterations)
  train_iter = [] 
  for i in range(len(train)):
    if(train[i][-1] in classes_current_iter):
      test.append(train[i]) 
      train_iter.append(train[i])

  train_loader = DataLoader(train_iter, shuffle = True, batch_size=batch_size, num_workers=2)

  valid_loader = DataLoader(test, shuffle = True, batch_size = batch_size, num_workers=2)

  incrementalFit(train_loader, valid_loader, resNet18)

  accuracy_train, preds_train, labels_train = test(train_loader, resNet18)
  accuracy_valid, preds_valid, labels_valid = test(valid_loader, resNet18)

  confusionMatrixData = confusion_matrix(preds_valid, labels_valid)
  plotConfusionMatrix(confusionMatrixData, 12)
    

  


1
1


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


1


ValueError: ignored

7